# Water corrosion prediction

Following are the python libraries used for data wrangling from the source file "Data requirement_Imperial Units.xlsx"

In [1]:
import pandas as pd
import numpy as np

On analysis, following pressure and temperatures were separated to generate individual tables to get predictions for each of the pipeline separately. so following divisions were done for convenience of the data wrangling

In [2]:
p1=["25PT1000 (psi)","25TT1000 (degF)","P1Water Velocity (m/s)"]
p2=["25PT2000 (psi)","25TT2000 (degF)","P2Water Velocity (m/s)"]
p3=["25PT3000 (psi)","25TT3000 (degF)","P3Water Velocity (m/s)"]
p4=["25PT4002 (psi)","25TT14002 (degF)","P4Water Velocity (m/s)"]
p14=["P1","P2","P3","P4","NaN"]
hs=['Corrossion Prediction','Operating Conditions','Design Data of Pipe','Pipe Material ','Pipeline Internal Coating']
sn=["Operating Conditions - Jan'18","Operating Conditions - June'18","Operating Conditions - Nov'18","Operating Conditions - April'19"]

Idea is to use monthly data and daily data for each of the table. so for final table columns are the features. Water parameters like ph, etc will become extra features. Monthly features are nealry 33 columns, and daily features are 102 columns. I have considered all the metrics given like valve positions, flows, pressures, temperatures. Total table will have 113 columns or features after removing unwanted or unspecified metrics.

In [3]:
def pmon(sn,p_day,k=["c,d","c,e","c,f"]):
    pdfs=[]
    for x in k:
        mon_df=pd.read_excel('/home/durga/Projects/water/Data requirement_Imperial Units.xlsx', sheet_name=sn, usecols=x,skiprows=5,nrows=33, header=None).set_index(2)
        mon_df=mon_df.T
        if x=="c,d":dc=p2+p3+p14+hs+p4
        elif x=="c,e": dc=p1+p3+p4+p14+hs
        elif x=="c,f": dc=p1+p2+p4+p14+hs
        replicate=pd.concat([mon_df]*p_day.shape[0],ignore_index=True)
        p_df=pd.concat([p_day,replicate],axis=1)
        columns_to_drop = list(set(dc) & set(p_df.columns))
        p_df.drop(columns_to_drop, axis=1,inplace=True )
        pdfs.append(p_df)
        
    return pdfs

def res_df(sn):
    dfs=[]
    for s in sn:
        p_day=pd.read_excel('/home/durga/Projects/water/Data requirement_Imperial Units.xlsx', sheet_name=s, usecols="c:ag",skiprows=39,nrows=102, header=None,).set_index(2)
        p_day=p_day.T
        p_day.reset_index(drop=True, inplace=True)
        dfs.extend(pmon(s,p_day))
    return dfs

dfs=res_df(sn)
dfs= [df.loc[:, df.columns.notnull()] for df in dfs]

Followng generates final tables with "Target Corrosion Depth" as the final varaible to be predicted. this is generated with interpolated values. so final table has 120 rows comes from 4 months of data for each pipeline. we can further combine the features of each pipeline to generate combined model. This is further to be ascertained.

In [5]:
num_sets = len(dfs) // 4
combined_dfs = []
for i in range(num_sets):
    set_indices = [i, i + num_sets, i + (2 * num_sets), i + (3 * num_sets)]
    set_dfs = [dfs[idx] for idx in set_indices if isinstance(dfs[idx], pd.DataFrame)]
    combined_df = pd.concat(set_dfs, axis=0)
    combined_dfs.append(combined_df)

for df in combined_dfs:
    numbers = df['Corrosion Depth'].unique().tolist()
    differences = [j - i for i, j in zip(numbers[:-1], numbers[1:])]
    average_difference = sum(differences) / len(differences)
    numbers.append(numbers[-1]+ round(average_difference,4))
    interpolated_values = []
    for i in range(len(numbers) - 1):
        interpolated_values += list(np.linspace(numbers[i], numbers[i+1], 30))
    df['Target Corrosion Depth'] = interpolated_values

pipeline 1,2,3 are as follows

In [8]:
combined_dfs[0]

2,Pressure Conversion,Temp Conversion,Temp conversion,25PT1000 (psi),25TT1000 (degF),P1Water Velocity (m/s),25M-1100A,25M-1100B,25M-2100A,25M-2100B,...,Age of Pipe,Pipe ID,Pipe Wall Thickness (mm),"Design Life, DL",Pipe Pressure (psi),Pipe Temperature (degF),Water Velocity (m/s),Corrosion Depth,Corrosion Rate,Target Corrosion Depth
0,14.5,1.8,32,41.905,51.8,1,1,0,1,0,...,15,0.386,6,20,38.135,48.584,1.003333,4.72,0.339,4.720000
1,14.5,1.8,32,42.775,50,0.8,1,0,1,0,...,15,0.386,6,20,38.135,48.584,1.003333,4.72,0.339,4.725517
2,14.5,1.8,32,44.95,48.2,0.9,1,0,1,0,...,15,0.386,6,20,38.135,48.584,1.003333,4.72,0.339,4.731034
3,14.5,1.8,32,41.905,50.9,1.1,1,0,1,0,...,15,0.386,6,20,38.135,48.584,1.003333,4.72,0.339,4.736552
4,14.5,1.8,32,42.775,52.34,1.2,1,0,1,0,...,15,0.386,6,20,38.135,48.584,1.003333,4.72,0.339,4.742069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,14.5,1.8,32,44.95,52.7,1,1,0,1,0,...,15,0.386,6,20,35.602333,48.83,1.003333,5.219,0.522875,5.362362
26,14.5,1.8,32,39.15,51.8,0.8,1,0,1,0,...,15,0.386,6,20,35.602333,48.83,1.003333,5.219,0.522875,5.368097
27,14.5,1.8,32,36.25,50,0.9,1,0,1,0,...,15,0.386,6,20,35.602333,48.83,1.003333,5.219,0.522875,5.373831
28,14.5,1.8,32,31.9,46.4,1.1,1,0,1,0,...,15,0.386,6,20,35.602333,48.83,1.003333,5.219,0.522875,5.379566


In [9]:
combined_dfs[1]

2,Pressure Conversion,Temp Conversion,Temp conversion,25PT2000 (psi),25TT2000 (degF),P2Water Velocity (m/s),25M-1100A,25M-1100B,25M-2100A,25M-2100B,...,Age of Pipe,Pipe ID,Pipe Wall Thickness (mm),"Design Life, DL",Pipe Pressure (psi),Pipe Temperature (degF),Water Velocity (m/s),Corrosion Depth,Corrosion Rate,Target Corrosion Depth
0,14.5,1.8,32,42.775,52.34,0.9,1,0,1,0,...,10,0.386,6,20,41.967833,49.766,0.884,2.354,0.212,2.354000
1,14.5,1.8,32,43.21,49.82,0.95,1,0,1,0,...,10,0.386,6,20,41.967833,49.766,0.884,2.354,0.212,2.357655
2,14.5,1.8,32,43.21,48.38,0.8,1,0,1,0,...,10,0.386,6,20,41.967833,49.766,0.884,2.354,0.212,2.361310
3,14.5,1.8,32,40.89,50.72,0.85,1,0,1,0,...,10,0.386,6,20,41.967833,49.766,0.884,2.354,0.212,2.364966
4,14.5,1.8,32,40.6,52.16,0.92,1,0,1,0,...,10,0.386,6,20,41.967833,49.766,0.884,2.354,0.212,2.368621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,14.5,1.8,32,42.775,52.52,0.9,1,0,1,0,...,10,0.386,6,20,42.040333,51.032,0.884,2.719,0.412,2.823914
26,14.5,1.8,32,43.21,51.62,0.95,1,0,1,0,...,10,0.386,6,20,42.040333,51.032,0.884,2.719,0.412,2.828110
27,14.5,1.8,32,43.21,49.82,0.8,1,0,1,0,...,10,0.386,6,20,42.040333,51.032,0.884,2.719,0.412,2.832307
28,14.5,1.8,32,41.18,50,0.85,1,0,1,0,...,10,0.386,6,20,42.040333,51.032,0.884,2.719,0.412,2.836503


In [11]:
combined_dfs[2]

2,Pressure Conversion,Temp Conversion,Temp conversion,25PT3000 (psi),25TT3000 (degF),P3Water Velocity (m/s),25M-1100A,25M-1100B,25M-2100A,25M-2100B,...,Age of Pipe,Pipe ID,Pipe Wall Thickness (mm),"Design Life, DL",Pipe Pressure (psi),Pipe Temperature (degF),Water Velocity (m/s),Corrosion Depth,Corrosion Rate,Target Corrosion Depth
0,14.5,1.8,32,43.355,52.16,1,1,0,1,0,...,13,0.386,6,20,42.369,49.802,1.2,3.025,0.35,3.025000
1,14.5,1.8,32,44.95,50,1.3,1,0,1,0,...,13,0.386,6,20,42.369,49.802,1.2,3.025,0.35,3.031034
2,14.5,1.8,32,43.355,48.56,1.4,1,0,1,0,...,13,0.386,6,20,42.369,49.802,1.2,3.025,0.35,3.037069
3,14.5,1.8,32,40.6,50.54,1.1,1,0,1,0,...,13,0.386,6,20,42.369,49.802,1.2,3.025,0.35,3.043103
4,14.5,1.8,32,40.455,51.98,1.2,1,0,1,0,...,13,0.386,6,20,42.369,49.802,1.2,3.025,0.35,3.049138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,14.5,1.8,32,43.355,52.7,1,1,0,1,0,...,13,0.386,6,20,42.4995,51.11,1.2,3.471,0.4325,3.599190
26,14.5,1.8,32,44.95,51.44,1.3,1,0,1,0,...,13,0.386,6,20,42.4995,51.11,1.2,3.471,0.4325,3.604317
27,14.5,1.8,32,43.355,49.64,1.4,1,0,1,0,...,13,0.386,6,20,42.4995,51.11,1.2,3.471,0.4325,3.609445
28,14.5,1.8,32,41.615,51.44,1.1,1,0,1,0,...,13,0.386,6,20,42.4995,51.11,1.2,3.471,0.4325,3.614572


Please find the attached excel sheet for the final results

In [6]:

excel_writer = pd.ExcelWriter('output.xlsx', engine='openpyxl')

for i, df in enumerate(combined_dfs, start=1):
    df.to_excel(excel_writer, sheet_name=f"p{i}", index=False)

excel_writer.close()